In [ ]:
# default_exp core

# VAT

> wip...

In [ ]:
#export
from fastai.basics import *
from fastai.test_utils import synth_learner

In [ ]:
#export
class ALUMCallback(Callback):
    "ALUM callback (draft)"
    run_valid=False

    def __init__(self, m, start_iter=None): #?? potentially start in the middle of training
        self.m = m

    def after_loss(self):
        #TODO: detach as appropriate
        old_weight = self.m.weight.data
        logits = self.pred
        print(f'{self.train_iter:2} - Do stuff here with weights of shape {old_weight.shape} and logits {logits.shape} and modify loss {self.loss:.4f}')
        # do ALUM stuff here


In [ ]:
model = nn.Sequential(
    nn.Linear(1,10, bias=False),
    nn.Linear(10,1, bias=False)
)
learn = synth_learner(model=model, cbs=ALUMCallback(model[0]))

In [ ]:
learn.show_training_loop()

Start Fit
   - before_fit     : [TrainEvalCallback, Recorder]
  Start Epoch Loop
     - before_epoch   : [Recorder]
    Start Train
       - before_train   : [TrainEvalCallback, Recorder]
      Start Batch Loop
         - before_batch   : []
         - after_pred     : []
         - after_loss     : [ALUMCallback]
         - before_backward: []
         - before_step    : []
         - after_step     : []
         - after_cancel_batch: []
         - after_batch    : [TrainEvalCallback, Recorder]
      End Batch Loop
    End Train
     - after_cancel_train: [Recorder]
     - after_train    : [Recorder]
    Start Valid
       - before_validate: [TrainEvalCallback, Recorder]
      Start Batch Loop
         - **CBs same as train batch**: []
      End Batch Loop
    End Valid
     - after_cancel_validate: [Recorder]
     - after_validate : [Recorder]
  End Epoch Loop
   - after_cancel_epoch: []
   - after_epoch    : [Recorder]
End Fit
 - after_cancel_fit: []
 - after_fit      : []


In [ ]:
learn.fit(1, 1e-3)

 0 - Do stuff here with weights of shape torch.Size([10, 1]) and logits torch.Size([16, 1]) and modify loss 13.3856
 1 - Do stuff here with weights of shape torch.Size([10, 1]) and logits torch.Size([16, 1]) and modify loss 14.8628
 2 - Do stuff here with weights of shape torch.Size([10, 1]) and logits torch.Size([16, 1]) and modify loss 8.4519
 3 - Do stuff here with weights of shape torch.Size([10, 1]) and logits torch.Size([16, 1]) and modify loss 16.2656
 4 - Do stuff here with weights of shape torch.Size([10, 1]) and logits torch.Size([16, 1]) and modify loss 13.1633
 5 - Do stuff here with weights of shape torch.Size([10, 1]) and logits torch.Size([16, 1]) and modify loss 10.4453
 6 - Do stuff here with weights of shape torch.Size([10, 1]) and logits torch.Size([16, 1]) and modify loss 10.9737
 7 - Do stuff here with weights of shape torch.Size([10, 1]) and logits torch.Size([16, 1]) and modify loss 14.0253
 8 - Do stuff here with weights of shape torch.Size([10, 1]) and logits t